# Imports

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import namedtuple

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF

import scipy.sparse as sp
from scipy.sparse import coo_matrix, csr_matrix, hstack
from scipy.sparse.linalg import norm